# Custom satellites

In this example we'll change two most important parameters: inclination & altitude (by modifying mean motion).

![orbital elements](https://upload.wikimedia.org/wikipedia/commons/thumb/e/eb/Orbit1.svg/266px-Orbit1.svg.png)

Full documentation of satellite model parameters can be found in [Skyfield's EarthSatellite documentation](http://rhodesmill.org/skyfield/api-satellites.html#skyfield.sgp4lib.EarthSatellite)

In [1]:
import os
from skyfield.api import load
import numpy as np
from skyfield.api import load, EarthSatellite

## Load existing satellite from NORAD database

In [2]:
resource_url = 'http://celestrak.com/NORAD/elements/resource.txt'

fname = 'tle.txt'
if os.path.isfile(fname):
    os.remove(fname)

satellites = load.tle(resource_url, filename=fname, reload=True)
satellite = satellites['HODOYOSHI-1']
print(satellite)

[#################################] 100% tle.txt


EarthSatellite 'HODOYOSHI-1' number=40299 epoch=2018-08-08T19:50:07Z


In [3]:
print(satellite.epoch.utc_jpl())

A.D. 2018-Aug-08 19:50:07.0719 UT


## Modify parameters

In [4]:
print('Inclination: {:}'.format(180 - np.rad2deg(satellite.model.inclo)))
print('Mean motion: {:}'.format(satellite.model.no))

Inclination: 82.6238
Mean motion: 0.06649628005841829


### Change inclination

In [5]:
# change inclination to 15 degrees
deg = 15
satellite.model.inclo = np.deg2rad(180 - deg)

### Change altitude

Altitude is very strongly related to mean motion, which is one of the orbital elements.

To change it we need to know what is orbital period of a satellite.

$\large n={\frac {2\pi }{P}}$

$n$ - mean motion in radians per second

$P = 2\pi \sqrt{\frac{a^{3}}{\mu}}$

$a$ - major semi axis

$\mu = GM_{body}$ - Standard gravitational parameter

$G$ - Gravitational constant ($G \approx 6.674*10^{-11} N \cdot kg^{-2} \cdot m^{2}$)

$M_{body}$ - Body mass in kilograms



In [6]:
G_CONST = 6.674 * 10**-11
EARTH_MASS = 5.97237 * 10**24
EARTH_RADIUS = 6.371 * 10**6
MU_EARTH = G_CONST * EARTH_MASS

def orbital_period(mu_body, a):
    return 2 * np.pi * np.sqrt(np.power(a, 3) / mu_body)

# change altitude to 400 km (above Earth's surface)
a = 400
P = orbital_period(MU_EARTH, EARTH_RADIUS + (a * 10**3))
no = (np.pi * 2) / P # rad / s

satellite.model.no = no * 60 # rad / min

In [7]:
print('Orbital period: {:0.0f}h {:0.0f}m'.format(P // 3600, ((P % 3600) * 60) / 3600))
print('Revolutions per day: {:0.1f}'.format((3600 * 24) / P))

Orbital period: 1h 32m
Revolutions per day: 15.6


## Review parameters

In [8]:
print('Inclination: {:}'.format(180 - np.rad2deg(satellite.model.inclo)))
print('Mean motion: {:}'.format(satellite.model.no))

Inclination: 15.0
Mean motion: 0.06798897338362271


## Sources

- http://rhodesmill.org/skyfield/api-satellites.html#skyfield.sgp4lib.EarthSatellite
- https://en.wikipedia.org/wiki/Two-line_element_set
- https://en.wikipedia.org/wiki/Orbital_speed
- https://en.wikipedia.org/wiki/Mean_motion